<a href="https://colab.research.google.com/github/disha4u/RAG-SS_Instructional_Aide/blob/main/Survey_Grader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Grading Survey Questions
This is a script for automatically grading survey questions for Software Security, one of the key tasks I had to do as an Instructional Aide for the subject. Here we use a basic minimum length check and quantized LLAMA model for verifying the answers.

In [ ]:
!pip install openpyxl

In [ ]:
!pip install gdown

In [ ]:
!pip install sentencepiece==0.1.99
!pip install transformers==4.31.0
!pip install accelerate==0.21.0
!pip install bitsandbytes==0.41.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from transformers import LlamaForCausalLM, LlamaTokenizer
import openpyxl

In [ ]:
df=pd.read_excel("Survey_ Module 6 Survey Student Analysis Report.xlsx" )

In [ ]:
#df.head(2)

In [ ]:
df.rename(columns=lambda x: str(x),inplace=True)

In [ ]:
#df.columns

In [ ]:
# for creating questions only excel used in the survey answers script.
#df2=df[["name","id","""19939725: Directions\nRead each question, reflect, and respond accordingly. To earn full credit, your answer should be a minimum of two (2) sentences, with at least one (1) sentence per question.\nHuman beings are magnificently curious creatures and designed to ask questions. Thinking about this past module:\n\nWhat is one (1) question you have not asked that you are curious about?\nWhy do you think you have not asked it until now?\n\nIf you do not have a question about the module, then please answer either of the question groups:\nGroup 1\n\nWhat is the name of a fictional character?\nWhat question do you think they would have about the module?\n\nFull-Credit Earning Example\nIn the movie Frozen II, the beloved snowman Olaf says he has a theory, “...about advancing technology is both our savior and our doom.” From the module, Olaf would ask, “Why do software developers not spend more time protecting their code from security threats?”\nGroup 2\n\nWhat is a hacking scene from movies, TV, books, etc.?\nWhat question would you ask the fictional characters from the scene about acts taken in the scene?\n\nFull-Credit Earning Example\nIn the movie Hackers, Kate hacks into Richard Gil's credit card company and invalidates the card. I would ask her how she bypassed the security of the credit card company's system."""]]
#df2.rename(columns={"19939725: Directions\nRead each question, reflect, and respond accordingly. To earn full credit, your answer should be a minimum of two (2) sentences, with at least one (1) sentence per question.\nHuman beings are magnificently curious creatures and designed to ask questions. Thinking about this past module:\n\nWhat is one (1) question you have not asked that you are curious about?\nWhy do you think you have not asked it until now?\n\nIf you do not have a question about the module, then please answer either of the question groups:\nGroup 1\n\nWhat is the name of a fictional character?\nWhat question do you think they would have about the module?\n\nFull-Credit Earning Example\nIn the movie Frozen II, the beloved snowman Olaf says he has a theory, “...about advancing technology is both our savior and our doom.” From the module, Olaf would ask, “Why do software developers not spend more time protecting their code from security threats?”\nGroup 2\n\nWhat is a hacking scene from movies, TV, books, etc.?\nWhat question would you ask the fictional characters from the scene about acts taken in the scene?\n\nFull-Credit Earning Example\nIn the movie Hackers, Kate hacks into Richard Gil's credit card company and invalidates the card. I would ask her how she bypassed the security of the credit card company's system.":"Original Question"}
#           ,inplace= True)
#df2.to_excel("Survey_ Module 6 Survey Student Analysis Report1.xlsx")

In [ ]:
 #columns which have to be graded
cols=["19939729","19939730","19939731"]

In [ ]:
#renaming them to just the numbers for ease of accessing them later
df.rename(columns=lambda x: x.split(":")[0] if x.startswith("19939729") or x.startswith("19939730") or x.startswith("19939731") else x, inplace=True)

In [ ]:
def check_length(txt):
  """
  This is to check number of words in the answer if less than 20/ 2 sentences approx we return 0 else 1
  :param txt:
  :return 0/1
  """
  l=len(txt.split())
  if l>20:
    return 1
  return 0

In [ ]:
def get_tokenizer(access_token,tokenizer_name="meta-llama/Llama-2-7b-chat-hf"):
  """
  :param access_token for llama model
  :param tokenizer_name default is Llama-2-7b-chat-hf
  :returns tokenizer
  """
  return LlamaTokenizer.from_pretrained(tokenizer_name, token=access_token)

def get_model(access_token,model_name="meta-llama/Llama-2-7b-chat-hf"):
  """
  :param access_token for llama model
  :param model_name default is Llama-2-7b-chat-hf
  :returns model
  """
  return LlamaForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto", token=access_token)

def classify(model,tokenizer,text,prompt=None):
  """
  classify the answer as matching the question or no
  :param model object to access the model
  :param tokenizer object to access the tokenizer
  :param text the answer text
  :param prompt, instructions to the model,  default is None
  :returns answer
  """
  if prompt==None:
    prompt="""You are given input just output the category , please dont give any explanation"""
  B_INST, E_INST = "[INST]", "[/INST]"
  input=f'{text}{B_INST}{prompt}{E_INST}'
  input_ids=tokenizer(input,return_tensors="pt")
  input_size=input_ids.input_ids.size()[1]
  generate_ids = model.generate(input_ids.input_ids.to(model.device),max_new_tokens=input_size+1,temperature=0.0)
  ans=tokenizer.decode(generate_ids.squeeze()[input_size+1:],skip_special_tokens=True)
  return(ans)

In [ ]:
def get_label(text):
  """
  Gives yes/no from the answer text
  :param text the answer text
  :returns answer
  """
  ans=text.split()[-1]
  return ans

In [ ]:
def normalize(text):
  """
  Normalizes the answer text, removes punctuation and extra spaces
  :param text the answer text
  :returns answer
  """
  text=text.lower()
  text=("").join(text.split("."))
  text=("").join(text.split(","))
  text=("").join(text.split())
  text=("").join(text.split("("))
  text=("").join(text.split(")"))
  text=("").join(text.split("'"))
  return text

In [ ]:
prompts=["""Given the above answer does it answer the question - What is a concept or idea that you learned in this module? .
The output should be yes or no , please dont give any explanation""",
         """Given the above answer does it answer the question - How do you feel about what you learned? .
The output should be yes or no , please dont give any explanation""",
        """Given the above answer does it answer the question -When might you apply what you have learned outside of this course? .
The output should be yes or no , please dont give any explanation""" ]

In [ ]:
llama_token="*******************************"
tokenizer=get_tokenizer(llama_token)
model=get_model(llama_token)

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
label=classify(model,tokenizer,df[cols[0]][0],prompts[0])

In [ ]:
label

'No.'

In [ ]:
l=[]

In [ ]:
#for all rows and questions , we check length and classify the answer as matching the question or not
#we append the row number and column to the list if we find answer not valid
for i in range(len(df)):
  for j,prompt in zip(cols,prompts):
    if type(df[j][i])!=str or df[j][i]=="":
      l.append((i,j))
      continue
    if check_length(df[j][i])!=1:
      l.append((i,j))
      continue
    label=classify(model,tokenizer,df[j][i],prompt)
    label=normalize(get_label(label))
    if label!="yes":
      l.append((i,j))

In [ ]:
#we print not valid answers
for i,j in l:
  print(i)
  print(j)
  print(df[j][i])
  print("----------------------------------------------------------------------------------------------------------------------------------------------")

0
19939729
nan
----------------------------------------------------------------------------------------------------------------------------------------------
0
19939730
nan
----------------------------------------------------------------------------------------------------------------------------------------------
0
19939731
nan
----------------------------------------------------------------------------------------------------------------------------------------------
16
19939731
In this module, I learned more about PHP as a programming language for dynamic websites and web based applications. I have never learned PHP, and felt that it was not as useful a language as other types for web programming. Learning some of the history of the language and it's features has helped me to have a better understanding of it's place in web technologies.I do not intend to learn any more about PHP, and will not likely use it outside of the course. I feel that several other languages have eclipsed PHP

In [ ]:
#we mark down the people who have not valid answers
for i,j in l:
  if i==69 or i==108 or i==114 or i==102:
    continue
  # if i==0 and j==19939723:
  #   continue
  df.loc[i, "score"] = df.loc[i, "score"] - 3

In [ ]:
for i,j in l:
  if i==69 or i==108 or i==114 or i==102:
    df.loc[i, "score"] = 10

In [ ]:
#save the new marks to the xl
df.to_excel("Survey_ Module 6 Survey Student Analysis Report_graded.xlsx", index=False)